# Pairs Trading Mean Reversion Strategy

## Account Connection with Interactive Brokers


In [1]:
import pandas as pd
from ib_insync import Forex, IB, util
from statsmodels.tsa.stattools import adfuller, coint
import numpy as np
from scipy.optimize import minimize
from scipy.integrate import quad
import datetime as dt
import time
import nest_asyncio

nest_asyncio.apply()

# Connect to IBKR TWS or Gateway
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=1)

myAccount = 'DU9177866'  ## Needs to be changed with the ouwn account number

# Check account balance
account_summary = ib.accountSummary(account=myAccount)
cash_balance = [item for item in account_summary if item.tag == 'TotalCashValue'][0].value
print(f"Account Cash Balance: {cash_balance}")



/Users/matteorampazzo/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Account Cash Balance: 1001785.99


## Account Connection with Interactive Brokers


In [2]:
# Define pairs of currency assets
pairs = [
    ('EURUSD', 'GBPUSD'),  # EUR/USD and GBP/USD
    ('USDJPY', 'USDCAD'),  # USD/JPY and USD/CAD
    ('AUDUSD', 'USDCHF'),  # AUD/USD and USD/CHF
    ('NZDUSD', 'EURGBP'),  # NZD/USD and EUR/GBP
    ('EURGBP', 'USDJPY'),  # EUR/GBP and USD/JPY
    ('USDCHF', 'USDJPY'),  # USD/CHF and USD/JPY
    ('EURJPY', 'GBPJPY'),  # EUR/JPY and GBP/JPY
    ('AUDJPY', 'NZDJPY'),  # AUD/JPY and NZD/JPY
    ('EURCHF', 'GBPCHF'),  # EUR/CHF and GBP/CHF
    ('AUDNZD', 'EURNZD'),  # AUD/NZD and EUR/NZD
    ('USDSGD', 'USDHKD'),  # USD/SGD and USD/HKD
    ('CADJPY', 'CHFJPY'),  # CAD/JPY and CHF/JPY
    ('GBPAUD', 'GBPCAD'),  # GBP/AUD and GBP/CAD
    ('EURAUD', 'EURCAD'),  # EUR/AUD and EUR/CAD
    ('USDNOK', 'USDSEK'),  # USD/NOK and USD/SEK
    ('EURTRY', 'USDTRY'),  # EUR/TRY and USD/TRY
    ('EURGBP', 'AUDJPY'),  # EUR/GBP and AUD/JPY
    ('EURGBP', 'GBPJPY'),  # EUR/GBP and GBP/JPY
    ('EURGBP', 'CHFJPY'),  # EUR/GBP and CHF/JPY
    ('EURGBP', 'USDSGD'),  # EUR/GBP and USD/SGD
    ('EURGBP', 'CADJPY'),  # EUR/GBP and CAD/JPY
    ('EURGBP', 'EURJPY'),  # EUR/GBP and EUR/JPY
    ('EURGBP', 'GBPCAD'),  # EUR/GBP and GBPCAD
    ('EURGBP', 'NZDJPY'),  # EUR/GBP and NZD/JPY
    ('AUDJPY', 'GBPJPY'),  # AUD/JPY and GBP/JPY
    ('AUDJPY', 'CHFJPY'),  # AUD/JPY and CHF/JPY
    ('AUDJPY', 'USDSGD'),  # AUD/JPY and USD/SGD
    ('AUDJPY', 'CADJPY'),  # AUD/JPY and CAD/JPY
    ('AUDJPY', 'EURJPY'),  # AUD/JPY and EUR/JPY
    ('AUDJPY', 'GBPCAD'),  # AUD/JPY and GBPCAD
    ('AUDJPY', 'USDJPY'),  # AUD/JPY and USD/JPY
    ('GBPJPY', 'CHFJPY'),  # GBP/JPY and CHF/JPY
    ('GBPJPY', 'USDSGD'),  # GBP/JPY and USD/SGD
    ('GBPJPY', 'CADJPY'),  # GBP/JPY and CAD/JPY
    ('GBPJPY', 'EURJPY'),  # GBP/JPY and EUR/JPY
    ('GBPJPY', 'GBPCAD'),  # GBP/JPY and GBPCAD
    ('GBPJPY', 'USDJPY'),  # GBP/JPY and USD/JPY
    ('CHFJPY', 'USDSGD'),  # CHF/JPY and USD/SGD
    ('CHFJPY', 'CADJPY'),  # CHF/JPY and CAD/JPY
    ('CHFJPY', 'EURJPY'),  # CHF/JPY and EUR/JPY
    ('CHFJPY', 'GBPCAD'),  # CHF/JPY and GBPCAD
    ('CHFJPY', 'NZDJPY'),  # CHF/JPY and NZD/JPY
    ('USDSGD', 'CADJPY'),  # USD/SGD and CAD/JPY
    ('USDSGD', 'EURJPY'),  # USD/SGD and EUR/JPY
    ('USDSGD', 'GBPCAD'),  # USD/SGD and GBPCAD
    ('CADJPY', 'EURJPY'),  # CAD/JPY and EUR/JPY
    ('CADJPY', 'GBPCAD'),  # CAD/JPY and GBPCAD
    ('CADJPY', 'NZDJPY'),  # CAD/JPY and NZD/JPY
    ('EURJPY', 'GBPCAD'),  # EUR/JPY and GBPCAD
    ('EURJPY', 'USDJPY'),  # EUR/JPY and USD/JPY
    ('GBPCAD', 'USDJPY'),  # GBP/CAD and USD/JPY
    ('GBPJPY', 'NZDJPY'),  # GBP/JPY and NZD/JPY
    ('USDJPY', 'NZDJPY')   # USD/JPY and NZD/JPY
]

# Define the start and end dates for downloading data
start_date = '20240101 00:00:00'
end_date = '20240531 23:59:59'

# Function to fetch historical data from IBKR
def fetch_historical_data(contract, start_date, end_date, bar_size='1 hour'):
    df_list = []
    current_end_date = dt.datetime.strptime(end_date, '%Y%m%d %H:%M:%S')

    while True:
        try:
            bars = ib.reqHistoricalData(
                contract,
                endDateTime=current_end_date.strftime('%Y%m%d %H:%M:%S'),
                durationStr='1 M',  # Fetch 1 month of data at a time
                barSizeSetting=bar_size,
                whatToShow='MIDPOINT',
                useRTH=False,
                formatDate=1
            )
            if not bars:
                break
            df = util.df(bars)
            df_list.append(df)
            # Subtract one second and ensure naive datetime
            current_end_date = df['date'].iloc[0].to_pydatetime().replace(tzinfo=None) - dt.timedelta(seconds=1)
            time.sleep(1)  # Respect IBKR API rate limits
            if current_end_date <= dt.datetime.strptime(start_date, '%Y%m%d %H:%M:%S'):
                break
        except Exception as e:
            print(f"Error fetching data for {contract.symbol}: {e}")
            break

    if df_list:
        return pd.concat(df_list).sort_index()
    else:
        return None

# Extract unique tickers from pairs
unique_tickers = set(ticker for pair in pairs for ticker in pair)

print("Initialization complete and ready for ADF test.")

Initialization complete and ready for ADF test.


## ADF Test

In [3]:
# Function to perform the ADF test and print the results
def adf_test(series, name):
    series = series.dropna()
    result = adfuller(series)
    test_statistic = f'{result[0]:.3f}'
    p_value = f'{result[1]:.6f}'  # Display p-value with higher precision
    critical_values_str = ', '.join([f'{key}: {value:.3f}' for key, value in result[4].items()])

    # Determine if the series is stationary
    status = 'stationary' if result[1] < 0.05 else 'non-stationary'

    # Print the result
    print(f'{name} Test Statistic: {test_statistic}, p-value: {p_value}. Critical Values: {critical_values_str}. The series is likely {status}.')

# Fetch and perform the ADF test for each ticker
for ticker in unique_tickers:
    print(f"Fetching data for {ticker}...")
    contract = Forex(ticker)
    data = fetch_historical_data(contract, start_date, end_date, bar_size='1 hour')
    if data is not None:
        print(f"Performing ADF test for {ticker}...")
        adf_test(data['close'], ticker)
    else:
        print(f"No data fetched for {ticker}")

print("ADF test complete.")

Fetching data for AUDUSD...
Performing ADF test for AUDUSD...
AUDUSD Test Statistic: -3.851, p-value: 0.002425. Critical Values: 1%: -3.432, 5%: -2.862, 10%: -2.567. The series is likely stationary.
Fetching data for EURGBP...
Performing ADF test for EURGBP...
EURGBP Test Statistic: -2.639, p-value: 0.085131. Critical Values: 1%: -3.432, 5%: -2.862, 10%: -2.567. The series is likely non-stationary.
Fetching data for USDCAD...
Performing ADF test for USDCAD...
USDCAD Test Statistic: -3.108, p-value: 0.025954. Critical Values: 1%: -3.432, 5%: -2.862, 10%: -2.567. The series is likely stationary.
Fetching data for USDCHF...
Performing ADF test for USDCHF...
USDCHF Test Statistic: -3.503, p-value: 0.007920. Critical Values: 1%: -3.432, 5%: -2.862, 10%: -2.567. The series is likely stationary.
Fetching data for USDJPY...
Performing ADF test for USDJPY...
USDJPY Test Statistic: -1.570, p-value: 0.498531. Critical Values: 1%: -3.432, 5%: -2.862, 10%: -2.567. The series is likely non-stationar

## CADF test

In [4]:
########################################## CHAT GPT-4 ##########################################
## Create using Chat GPT-4, explaining the concept and ask for suggestion on how to make the code

# Function to perform the CADF test and print the results
def cadf_test(series1, series2, name1, name2):
    series1 = series1.dropna()
    series2 = series2.dropna()

    # Ensure both series have the same length
    if len(series1) != len(series2):
        min_len = min(len(series1), len(series2))
        series1 = series1[-min_len:]
        series2 = series2[-min_len:]

    score, p_value, critical_values = coint(series1, series2)
    test_statistic = f'{score:.3f}'
    p_value_formatted = f'{p_value:.6f}'

    critical_values_str = ', '.join([f'{key}: {value:.3f}' for key, value in zip(['1%', '5%', '10%'], critical_values)])

    # Determine if the series are cointegrated
    status = 'cointegrated' if p_value < 0.05 else 'not cointegrated'

    # Print the result
    print(f'{name1} & {name2} Test Statistic: {test_statistic}, p-value: {p_value_formatted}. The series are likely {status}.')


# Perform the CADF test for each pair
for name1, name2 in pairs:
    print(f"Fetching data for {name1} and {name2}...")
    contract1 = Forex(name1)
    contract2 = Forex(name2)
    
    data1 = fetch_historical_data(contract1, start_date, end_date, bar_size='1 hour')
    data2 = fetch_historical_data(contract2, start_date, end_date, bar_size='1 hour')
    
    if data1 is not None and data2 is not None:
        print(f"Performing CADF test for {name1} and {name2}...")
        cadf_test(data1['close'], data2['close'], name1, name2)
    else:
        if data1 is None:
            print(f"No data fetched for {name1}")
        if data2 is None:
            print(f"No data fetched for {name2}")

print("CADF test complete.")

Fetching data for EURUSD and GBPUSD...
Performing CADF test for EURUSD and GBPUSD...
EURUSD & GBPUSD Test Statistic: -2.688, p-value: 0.203837. The series are likely not cointegrated.
Fetching data for USDJPY and USDCAD...
Performing CADF test for USDJPY and USDCAD...
USDJPY & USDCAD Test Statistic: -0.889, p-value: 0.921890. The series are likely not cointegrated.
Fetching data for AUDUSD and USDCHF...
Performing CADF test for AUDUSD and USDCHF...
AUDUSD & USDCHF Test Statistic: -3.590, p-value: 0.025182. The series are likely cointegrated.
Fetching data for NZDUSD and EURGBP...
Performing CADF test for NZDUSD and EURGBP...
NZDUSD & EURGBP Test Statistic: -5.162, p-value: 0.000082. The series are likely cointegrated.
CADF test complete.


In [5]:
# Disconnect from IBKR
ib.disconnect()

# **** BACKTESTING ****
In this section, we will backtest our strategy using optimal timing strategies for trading a mean-reverting price spread. We'll optimize the positions of AUDJPY and CADJPY to ensure the intraday portfolio value aligns closely with an Ornstein-Uhlenbeck (OU) process, using maximum likelihood estimation for the best fit.


### 1 - Account connection and Data acquisition¶

In [1]:
import nest_asyncio
from scipy.optimize import minimize
from ib_insync import *
import pandas as pd
import datetime as dt
import time
import numpy as np
from scipy.integrate import quad

# Apply nest_asyncio to avoid asyncio loop issues
nest_asyncio.apply()

# Connect to IBKR TWS or Gateway
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=1)

myAccount = 'DU9177866'

valuta1_contract = Forex('AUDJPY')
valuta2_contract = Forex('CADJPY')

# Function to fetch historical data for a month
def fetch_month_data(contract, start_date='20240501 00:00:00', end_date='20240531 23:59:59', bar_size='15 mins'):
    df_list = []
    current_end_date = dt.datetime.strptime(end_date, '%Y%m%d %H:%M:%S')

    while True:
        try:
            bars = ib.reqHistoricalData(
                contract,
                endDateTime=current_end_date.strftime('%Y%m%d %H:%M:%S'),
                durationStr='1 M',
                barSizeSetting=bar_size,
                whatToShow='MIDPOINT',
                useRTH=False,
                formatDate=1
            )
            if not bars:
                break
            df = util.df(bars)
            df_list.append(df)
            current_end_date = df['date'].iloc[0].to_pydatetime().replace(tzinfo=None) - dt.timedelta(seconds=1)
            time.sleep(1)
            if current_end_date <= dt.datetime.strptime(start_date, '%Y%m%d %H:%M:%S'):
                break
        except Exception as e:
            print(f"Error fetching data: {e}")
            break

    return pd.concat(df_list).sort_index() if df_list else None

# Fetch data for AUD/JPY and CAD/JPY
valuta1_data = fetch_month_data(valuta1_contract, start_date='20240501 00:00:00', end_date='20240531 23:59:59', bar_size='15 mins')
valuta2_data = fetch_month_data(valuta2_contract, start_date='20240501 00:00:00', end_date='20240531 23:59:59', bar_size='15 mins')

# Disconnect from IBKR
ib.disconnect()

# Ensure data is properly fetched
if valuta1_data is None or valuta2_data is None:
    raise ValueError("Failed to fetch data for one or both contracts.")

# Print column names to debug
print("Columns in valuta1_data:", valuta1_data.columns)
print("Columns in valuta2_data:", valuta2_data.columns)

# Convert the 'date' column to datetime and set as index
valuta1_data['date'] = pd.to_datetime(valuta1_data['date'])
valuta2_data['date'] = pd.to_datetime(valuta2_data['date'])
valuta1_data.set_index('date', inplace=True)
valuta2_data.set_index('date', inplace=True)

# Resample to 15-minute intervals
valuta1_resampled = valuta1_data.resample('15min').last().dropna()
valuta2_resampled = valuta2_data.resample('15min').last().dropna()

# Merge datasets
merged_data = pd.merge(valuta1_resampled, valuta2_resampled, left_index=True, right_index=True, suffixes=('_valuta1', '_valuta2'))
merged_data.dropna(inplace=True)

# Extract price arrays
valuta1_prices = merged_data['close_valuta1'].values
valuta2_prices = merged_data['close_valuta2'].values

print("Data fetching and preparation complete.")


/Users/matteorampazzo/anaconda3/lib/python3.11/site-packages/pandas/core/arrays/masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Columns in valuta1_data: Index(['date', 'open', 'high', 'low', 'close', 'volume', 'average',
       'barCount'],
      dtype='object')
Columns in valuta2_data: Index(['date', 'open', 'high', 'low', 'close', 'volume', 'average',
       'barCount'],
      dtype='object')
Data fetching and preparation complete.


### 2 - Optimization functions

In [2]:
# Function to construct portfolio
def construct_portfolio(valuta1_prices, valuta2_prices, B):
    return valuta1_prices - B * valuta2_prices

# Function to optimize pair ratio
def optimize_pair_ratio(valuta1_prices, valuta2_prices):
    def negative_log_likelihood(B):
        portfolio_values = construct_portfolio(valuta1_prices, valuta2_prices, B)
        portfolio_values = (portfolio_values - np.mean(portfolio_values)) / np.std(portfolio_values)
        _, theta, _ = optimize_ou_parameters(portfolio_values)
        return -theta
    result = minimize(negative_log_likelihood, 0.0, bounds=[(-2, 2)])
    B_opt = result.x[0]
    return B_opt

# Function to optimize OU parameters
def optimize_ou_parameters(portfolio_values):
    def ou_log_likelihood(params, X):
        mu, theta, sigma = params
        n = len(X)
        dt = 1
        X_diff = np.diff(X)
        X_prev = X[:-1]
        likelihood = (
            -n / 2 * np.log(2 * np.pi * sigma**2 * (1 - np.exp(-2 * mu * dt)) / (2 * mu))
            - (X_diff - (theta - X_prev) * (1 - np.exp(-mu * dt)))**2
            / (2 * sigma**2 * (1 - np.exp(-2 * mu * dt)) / (2 * mu))
        )
        return -np.sum(likelihood)
    
    initial_params = [0.1, portfolio_values.mean(), portfolio_values.std()]
    bounds = [(1e-6, 1), (None, None), (1e-6, 1)]
    result = minimize(ou_log_likelihood, initial_params, args=(portfolio_values,), bounds=bounds)
    print(f"Optimized OU parameters: {result.x}")
    return result.x

print("Optimization functions ready.")

Optimization functions ready.


### 3 - Optimal exit level

In [3]:
def calculate_optimal_exit_level(theta, mu, sigma, c=0):
    if sigma < 1e-6 or mu < 1e-6:
        raise ValueError("Sigma or mu is too small, causing numerical instability in calculations.")
    
    def F(x):
        integral_result, _ = quad(
            lambda u: (u**(2*mu/sigma**2 - 1)) * np.exp(np.sqrt(2*mu/sigma**2) * (theta - x) * u - u**2 / 2), 
            0, 
            np.inf
        )
        return integral_result
    
    def F_prime(x):
        integral_result, _ = quad(
            lambda u: (u**(2*mu/sigma**2 - 2)) * np.exp(np.sqrt(2*mu/sigma**2) * (theta - x) * u - u**2 / 2) 
            * (np.sqrt(2*mu/sigma**2) * (theta - x) - u), 
            0, 
            np.inf
        )
        return integral_result
    
    def objective(b):
        return np.abs(F(b) - (b - c) * F_prime(b))
    
    result = minimize(objective, theta, bounds=[(theta - 5 * sigma, theta + 5 * sigma)])
    return result.x[0]

print("Optimal exit level calculation function ready.")

Optimal exit level calculation function ready.


### 4 - Optimal exit signals

In [4]:
########################################## CHAT GPT-4 ##########################################
## Create using Chat GPT-4, explaining the concept and ask for suggestion on how to make the code


def calculate_optimal_exit_signals(portfolio_values, optimal_exit_level, window_size=30):
    signals = []
    for i in range(window_size, len(portfolio_values)):
        current_value = portfolio_values[i]
        moving_avg = np.mean(portfolio_values[i - window_size:i])
        std_dev = np.std(portfolio_values[i - window_size:i])
        if current_value < moving_avg - std_dev:
            signals.append(1)
        elif current_value > optimal_exit_level:
            signals.append(-1)
        else:
            signals.append(0)
    signals = [0] * window_size + signals
    return np.array(signals)

def simulate_optimal_exit_strategy(portfolio_values, signals, initial_cash=100000, trade_amount=50000):
    position = 0
    cash = initial_cash
    portfolio = []
    trade_log = []  # To log the trades
    min_length = min(len(signals), len(portfolio_values))
    for i in range(min_length):
        signal = signals[i]
        current_value = portfolio_values[i]
        if signal == 1 and position == 0:
            position = trade_amount / current_value
            cash -= trade_amount
            trade_log.append((i, 'BUY', current_value))
        elif signal == -1 and position > 0:
            cash += position * current_value
            position = 0
            trade_log.append((i, 'SELL', current_value))
        portfolio_value = cash + (position * current_value)
        portfolio.append(portfolio_value)
    return np.array(portfolio), trade_log

print("Signal generation and strategy simulation functions ready.")

Signal generation and strategy simulation functions ready.


### 5 - Optimized parameters and trade

In [5]:
########################################## CHAT GPT-4 ##########################################
## Create using Chat GPT-4, explaining the concept and ask for suggestion on how to make the code


# Optimize pair ratio
B_opt = optimize_pair_ratio(valuta1_prices, valuta2_prices)
print(f"Optimized pair ratio B: {B_opt:.4f}")

# Construct the portfolio
portfolio_values = construct_portfolio(valuta1_prices, valuta2_prices, B_opt)

# Optimize the OU parameters
mu_opt, theta_opt, sigma_opt = optimize_ou_parameters(portfolio_values)
print(f"Optimized parameters: mu={mu_opt:.4f}, theta={theta_opt:.4f}, sigma={sigma_opt:.4f}")

# Calculate the optimal exit level
try:
    optimal_exit_level = calculate_optimal_exit_level(theta_opt, mu_opt, sigma_opt)
    print(f"Optimal exit level b: {optimal_exit_level:.4f}")
except ValueError as e:
    print(f"Error in calculating optimal exit level: {e}")
    optimal_exit_level = theta_opt  # Use theta as a fallback

# Calculate the optimal exit signals
optimal_exit_signals = calculate_optimal_exit_signals(portfolio_values, optimal_exit_level)

# Simulate the optimal exit strategy
portfolio_simulation, trade_log = simulate_optimal_exit_strategy(portfolio_values, optimal_exit_signals, initial_cash=100000, trade_amount=50000)

# Print the final portfolio value
final_value = portfolio_simulation[-1]
print(f"Final portfolio value: ${final_value:.2f}")

# Print the trade log
print("Trade Log:")
for trade in trade_log:
    trade_time, trade_type, trade_price = trade
    print(f"Time: {trade_time}, Trade: {trade_type}, Price: {trade_price:.2f}")

Optimized OU parameters: [ 1.00000000e-06 -5.31030954e+01  5.40587866e-04]
Optimized OU parameters: [ 1.00000000e-06 -5.37167994e+01  5.40588091e-04]
Optimized OU parameters: [ 1.00000000e-06 -4.14847312e+00  7.01854904e-04]
Optimized OU parameters: [ 1.00000000e-06 -4.14838231e+00  7.01854904e-04]
Optimized OU parameters: [ 1.00000000e-06 -2.89386261e+00  6.18759426e-04]
Optimized OU parameters: [ 1.00000000e-06 -2.56870017e+00  6.18764031e-04]
Optimized OU parameters: [ 1.00000000e-06 -4.65991579e+00  5.77891037e-04]
Optimized OU parameters: [4.99673933e-05 1.64121494e+01 5.77763365e-04]
Optimized OU parameters: [ 1.00000000e-06 -2.45298367e+00  5.62891064e-04]
Optimized OU parameters: [ 1.00000000e-06 -2.49287649e+00  5.62888038e-04]
Optimized OU parameters: [ 1.00000000e-06 -5.22388601e+00  5.47924604e-04]
Optimized OU parameters: [ 1.00000000e-06 -6.65606294e+00  5.47931352e-04]
Optimized OU parameters: [ 1.00000000e-06 -5.25230330e+00  5.41584544e-04]
Optimized OU parameters: [ 1

### 6 - Results

In [6]:
# Calculate daily returns
def calculate_daily_returns(portfolio_values):
    returns = np.diff(portfolio_values) / portfolio_values[:-1]
    return returns

# Calculate Sharpe ratio
def calculate_sharpe_ratio(returns, risk_free_rate=0.0):
    mean_return = np.mean(returns)
    std_return = np.std(returns)
    sharpe_ratio = (mean_return - risk_free_rate) / std_return
    return sharpe_ratio

# Calculate gain or loss for each trade and track win/loss performance
def calculate_trade_performance(trade_log):
    trade_performance = []
    for i in range(0, len(trade_log), 2):
        if i + 1 < len(trade_log):
            buy_trade = trade_log[i]
            sell_trade = trade_log[i + 1]
            gain_or_loss = (sell_trade[2] - buy_trade[2]) * 50000 / buy_trade[2]
            performance = 'Win' if gain_or_loss > 0 else 'Loss'
            trade_performance.append((buy_trade, sell_trade, gain_or_loss, performance))
    return trade_performance

# Calculate overall win rate
def calculate_win_rate(trade_performance):
    wins = sum(1 for trade in trade_performance if trade[3] == 'Win')
    total_trades = len(trade_performance)
    win_rate = wins / total_trades if total_trades > 0 else 0
    return win_rate

# Final summary
def final_summary(portfolio_values, trade_log):
    initial_value = portfolio_values[0]
    final_value = portfolio_values[-1]
    total_return = (final_value - initial_value) / initial_value
    daily_returns = calculate_daily_returns(portfolio_values)
    sharpe_ratio = calculate_sharpe_ratio(daily_returns)
    trade_performance = calculate_trade_performance(trade_log)
    win_rate = calculate_win_rate(trade_performance)

    print(f"Final portfolio value: ${final_value:.2f}")
    print(f"Total return: {total_return * 100:.2f}%")
    print(f"Sharpe ratio: {sharpe_ratio:.2f}")
    print(f"Win rate: {win_rate * 100:.2f}%")
    print("Trade Performance Log:")
    for trade in trade_performance:
        buy_trade, sell_trade, gain_or_loss, performance = trade
        print(f"Buy Time: {buy_trade[0]}, Sell Time: {sell_trade[0]}, Buy Price: {buy_trade[2]:.2f}, Sell Price: {sell_trade[2]:.2f}, Gain/Loss: {gain_or_loss:.2f}, Performance: {performance}")

# Print the final summary
final_summary(portfolio_simulation, trade_log)

Final portfolio value: $100905.95
Total return: 0.91%
Sharpe ratio: 0.01
Win rate: 81.21%
Trade Performance Log:
Buy Time: 106, Sell Time: 114, Buy Price: 98.67, Sell Price: 98.70, Gain/Loss: 14.95, Performance: Win
Buy Time: 115, Sell Time: 116, Buy Price: 98.66, Sell Price: 98.67, Gain/Loss: 7.86, Performance: Win
Buy Time: 139, Sell Time: 140, Buy Price: 98.68, Sell Price: 98.72, Gain/Loss: 23.82, Performance: Win
Buy Time: 146, Sell Time: 148, Buy Price: 98.72, Sell Price: 98.76, Gain/Loss: 20.26, Performance: Win
Buy Time: 273, Sell Time: 290, Buy Price: 100.19, Sell Price: 99.65, Gain/Loss: -271.72, Performance: Loss
Buy Time: 291, Sell Time: 293, Buy Price: 99.62, Sell Price: 99.63, Gain/Loss: 5.52, Performance: Win
Buy Time: 297, Sell Time: 311, Buy Price: 99.51, Sell Price: 99.39, Gain/Loss: -61.05, Performance: Loss
Buy Time: 348, Sell Time: 349, Buy Price: 99.42, Sell Price: 99.52, Gain/Loss: 53.06, Performance: Win
Buy Time: 350, Sell Time: 352, Buy Price: 99.42, Sell Price

In [7]:
print (B_opt, mu_opt, theta_opt, sigma_opt)

-3.2683111674280833e-06 1e-06 94.240639249039 0.0010409586950222967


## Live Trading

## Trading Strategy Explanation

The trading strategy implemented in the code follows these steps:

### Setup and Initialization:
- Connect to the Interactive Brokers (IBKR) API using `ib_insync`.
- Define the forex contracts for AUDJPY and CADJPY.
- Set optimized parameters from backtesting for the trading strategy.

### Fetching Recent Prices:
- Retrieve recent price data for AUDJPY and CADJPY using the `reqHistoricalData` method from IBKR.
- Ensure the data is properly fetched and converted into a pandas DataFrame.

### Calculating Optimal Entry and Exit Prices:
- **Optimal Entry Price**: Calculate the optimal entry price for trading AUDJPY based on the current prices of AUDJPY and CADJPY. This is done using the formula: `current_audusd_price - B_opt * current_usdchf_price`, where `B_opt` is a parameter from backtesting.
- **Optimal Exit Price**: Calculate the optimal exit price based on the `theta` parameter from backtesting. This is a placeholder and can be adjusted with more complex logic if needed.

### Trading Logic:
- **Determine Trade Action**: Based on the current price of  AUDJPY relative to the optimal entry price:
  - If the current price is higher than the optimal entry price, place a **BUY** limit order at the optimal entry price.
  - If the current price is lower than the optimal entry price, place a **SELL** limit order at the optimal entry price.
- **Place and Monitor Orders**: Place the limit order and monitor its status:
  - If the order is not filled within 5 minutes, cancel the order.
  - If the order is filled, monitor the price to determine the exit:
    - For a **BUY** order, place a **SELL** market order when the price reaches the optimal exit price or after 2 hours.
    - For a **SELL** order, place a **BUY** market order when the price reaches the optimal exit price or after 2 hours.

### Main Loop and Trading Hours:
- The main loop runs continuously during trading hours (7:00 AM to 11:00 PM).
- Check if it's within trading hours before attempting to place new trades.
- If outside trading hours, the script waits and checks again after 5 minutes.

### End-of-Day Processing:
- At the end of the trading day (after 11:00 PM), close all open positions.
- Calculate the daily return based on the balance at the start and end of the trading day:
  - Daily Return = ((End Balance - Start Balance) / Start Balance) * 100
- Print the daily return and disconnect from the IBKR API.

### Summary
This strategy aims to trade AUDJPY based on its relationship with CADJPY. It uses backtested parameters to determine optimal entry and exit points, places limit orders accordingly, and monitors trades to ensure they are executed or canceled as needed. The strategy runs continuously during specified trading hours and calculates the daily return at the end of each trading day.


In [52]:
import nest_asyncio
from ib_insync import *
from datetime import datetime, time, timedelta
from time import sleep

# Apply nest_asyncio to avoid asyncio loop issues
nest_asyncio.apply()

# Connect to IBKR TWS or Gateway
ib = IB()
connected = ib.connect('127.0.0.1', 7497, clientId=1)

myAccount = 'DU9177866'

# Check connection status
print(f"Connected: {connected}")

Connected: <IB connected to 127.0.0.1:7497 clientId=1>


In [53]:
# Define the contracts
valuta1_contract = Forex('AUDJPY')
valuta2_contract = Forex('CADJPY')

# Optimized parameters B_opt, mu_opt, theta_opt, sigma_opt comese from backtesting
# Make sure that the backtesting is done before running this script

In [54]:
########################################## CHAT GPT-4 ##########################################
## Create using Chat GPT-4, explaining the concept and ask for suggestion on how to make the code


# Request tracking
last_requests = []

def can_make_request():
    global last_requests
    now = datetime.now()
    # Remove requests older than 10 minutes
    last_requests = [req_time for req_time in last_requests if now - req_time < timedelta(minutes=10)]
    return len(last_requests) < 60

def get_recent_price(contract, duration='1800 S', bar_size='1 secs'):
    while True:
        if not can_make_request():
            print("Reached request limit. Sleeping for 15 seconds.")
            sleep(15)
            continue

        try:
            bars = ib.reqHistoricalData(
                contract,
                endDateTime='',
                durationStr=duration,
                barSizeSetting=bar_size,
                whatToShow='MIDPOINT',
                useRTH=True,
                formatDate=1
            )
            df = util.df(bars)
            if df is None or df.empty:
                raise ValueError(f"Failed to get historical data for {contract.symbol}")
            
            last_requests.append(datetime.now())
            price = df.iloc[-1]['close']
            print(f"Current {contract.symbol} Price: {price}")
            return price
        except Exception as e:
            if "pacing violation" in str(e).lower():
                print(f"Pacing violation error: {e}. Sleeping for 60 seconds.")
                sleep(60)
            else:
                print(f"An error occurred: {e}")
                sleep(5)

def calculate_optimal_entry(current_valuta1_price, current_valuta2_price):
    return round(current_valuta1_price - B_opt * current_valuta2_price, 3)

def calculate_optimal_exit(theta, mu, sigma):
    return round(theta, 3)  # Placeholder, replace with actual logic if different

def check_account_status():
    account_summary = ib.accountSummary()
    account_values = {av.tag: av.value for av in account_summary if av.account == myAccount}
    
    balance = float(account_values.get('NetLiquidation', 0))
    buying_power = float(account_values.get('BuyingPower', 0))
    selling_power = float(account_values.get('AvailableFunds', 0))
    
    print(f"Balance: {balance}")
    print(f"Buying Power: {buying_power}")
    print(f"Selling Power: {selling_power}")
    
    if balance < 50000 or buying_power < 50000 or selling_power < 50000:  # Example threshold values
        raise ValueError("Insufficient balance, buying power, or selling power to proceed with trading.")
    
    return balance

In [55]:
########################################## CHAT GPT-4 ##########################################
## Create using Chat GPT-4, explaining the concept and ask for suggestion on how to make the code


def place_and_monitor_order():
    while True:
        current_valuta1_price = get_recent_price(valuta1_contract)
        current_valuta2_price = get_recent_price(valuta2_contract)

        optimal_entry_price = calculate_optimal_entry(current_valuta1_price, current_valuta2_price)
        optimal_exit_price = calculate_optimal_exit(theta_opt, mu_opt, sigma_opt)

        print(f"Current Valuta1 Price: {current_valuta1_price}")
        print(f"Current Valuta2 Price: {current_valuta2_price}")
        print(f"Optimal Entry Price: {optimal_entry_price}")
        print(f"Optimal Exit Price: {optimal_exit_price}")

        trade_quantity = 50000  # Example trade quantity, adjust as needed

        # Determine the initial action based on entry and exit prices
        if optimal_entry_price < optimal_exit_price:
            action = 'BUY'
        else:
            action = 'SELL'

        if (action == 'BUY' and current_valuta1_price <= optimal_entry_price) or (action == 'SELL' and current_valuta1_price >= optimal_entry_price):
            print("Entry conditions not met. Restarting the cycle.")
            sleep(60)  # Sleep for 60 seconds before restarting the cycle
            continue

        limit_price = optimal_entry_price
        print(f"Action: {action}, Limit Price: {limit_price}")

        # Place the limit order for entry
        entry_order = LimitOrder(action, trade_quantity, limit_price, tif='DAY')
        entry_trade = ib.placeOrder(valuta1_contract, entry_order)

        print(f"Placed {action} limit order at {limit_price}")
        print(entry_trade)

        filled = False
        start_time = datetime.now()
        while (datetime.now() - start_time) < timedelta(minutes=5):
            if entry_trade.orderStatus.status == 'Filled':
                filled = True
                break
            ib.sleep(5)

        if not filled:
            ib.cancelOrder(entry_trade.order)
            print(f"Cancelled {action} limit order at {limit_price}")
            continue
        else:
            print(f"Order filled: {action} at {limit_price}")
            print(entry_trade)

            # Place a take profit order
            opposite_action = 'SELL' if action == 'BUY' else 'BUY'
            take_profit_order = LimitOrder(opposite_action, trade_quantity, optimal_exit_price, tif='DAY')
            take_profit_trade = ib.placeOrder(valuta1_contract, take_profit_order)
            print(f"Placed {opposite_action} take profit order at {optimal_exit_price}")
            print(take_profit_trade)

            entry_time = datetime.now()
            while (datetime.now() - entry_time) < timedelta(hours=2):
                current_valuta1_price = get_recent_price(valuta1_contract)
                print(f"Current Valuta1 Price: {current_valuta1_price}")  # Debugging
                if (action == 'BUY' and current_valuta1_price >= optimal_exit_price) or \
                   (action == 'SELL' and current_valuta1_price <= optimal_exit_price):
                    close_order = MarketOrder(opposite_action, trade_quantity)
                    close_trade = ib.placeOrder(valuta1_contract, close_order)
                    while not close_trade.isDone():
                        ib.waitOnUpdate()
                    print(f"Closed position with {opposite_action} order at {current_valuta1_price}")
                    print(close_trade)
                    break
                ib.sleep(60)  # Sleep for 60 seconds before checking the price again

            # If the position is still open after 2 hours, close it
            if (datetime.now() - entry_time) >= timedelta(hours=2):
                # Cancel the take profit order
                ib.cancelOrder(take_profit_order)
                print(f"Cancelled take profit order at {optimal_exit_price}")

                # Place market order to close the position
                close_order = MarketOrder(opposite_action, trade_quantity)
                close_trade = ib.placeOrder(valuta1_contract, close_order)
                while not close_trade.isDone():
                    ib.waitOnUpdate()
                print(f"Closed position after 2 hours with {opposite_action} market order at {current_valuta1_price}")
                print(close_trade)

def within_trading_hours():
    now = datetime.now().time()
    return time(7, 0) <= now <= time(23, 0)

In [56]:
# Main loop
start_balance = check_account_status()  # Initial balance at the start of the day
print(f"Start Balance: {start_balance}")

while True:
    if within_trading_hours():
        try:
            place_and_monitor_order()
        except Exception as e:
            print(f"An error occurred: {e}")
            ib.sleep(5)  # Wait before retrying
    else:
        print("Outside trading hours. Waiting...")
        sleep(300)  # Wait 5 minutes before checking again

        if datetime.now().time() > time(23, 0):
            positions = ib.positions(account=myAccount)
            for position in positions:
                contract = position.contract
                position_size = position.position
                if position_size > 0:
                    close_action = 'SELL'
                else:
                    close_action = 'BUY'
                close_order = MarketOrder(close_action, abs(position_size))
                ib.placeOrder(contract, close_order)
                print(f"Closed {close_action} position of {position_size} for {contract}")

            break

Balance: 1004168.64
Buying Power: 3992711.53
Selling Power: 998177.88
Start Balance: 1004168.64
Current AUD Price: 105.8045
Current CAD Price: 116.3415
Current Valuta1 Price: 105.8045
Current Valuta2 Price: 116.3415
Optimal Entry Price: 105.805
Optimal Exit Price: 94.241
Action: SELL, Limit Price: 105.805
Placed SELL limit order at 105.805
Trade(contract=Forex('AUDJPY', exchange='IDEALPRO'), order=LimitOrder(orderId=2230, clientId=1, action='SELL', totalQuantity=50000, lmtPrice=105.805, tif='DAY'), orderStatus=OrderStatus(orderId=2230, status='PendingSubmit', filled=0.0, remaining=0.0, avgFillPrice=0.0, permId=0, parentId=0, lastFillPrice=0.0, clientId=0, whyHeld='', mktCapPrice=0.0), fills=[], log=[TradeLogEntry(time=datetime.datetime(2024, 6, 21, 15, 42, 30, 950951, tzinfo=datetime.timezone.utc), status='PendingSubmit', message='', errorCode=0)], advancedError='')
Order filled: SELL at 105.805
Trade(contract=Forex('AUDJPY', exchange='IDEALPRO'), order=LimitOrder(orderId=2230, clientI

In [50]:
# Calculate the daily return after the trading day ends and all positions are closed
end_balance = check_account_status()  # Balance at the end of the day
print(f"End Balance: {end_balance}")

# Calculate the daily return
daily_return = (end_balance - start_balance) / start_balance * 100
print(f"Daily Return: {daily_return:.2f}%")

Balance: 1004168.64
Buying Power: 3992711.53
Selling Power: 998177.88
End Balance: 1004168.64
Daily Return: 0.00%


In [51]:
ib.disconnect()
